In [2]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_json("../../data/raw-old.json")
data.set_index("product_id", inplace=True)
len(data.columns)

57

In [ ]:
data.dtypes

mrp                                      int64
Brand                                   object
Model Name                              object
Network Service Provider                object
OS                                      object
Cellular Technology                     object
RAM                                     object
Product Dimensions                      object
Batteries                               object
Item model number                       object
Wireless communication technologies     object
Connectivity technologies               object
GPS                                     object
Special features                        object
Display technology                      object
Other display features                  object
Resolution                              object
Other camera features                   object
Audio Jack                              object
Form factor                             object
Colour                                  object
Battery Power

In [ ]:
data.columns

Index(['mrp', 'Brand', 'Model Name', 'Network Service Provider', 'OS',
       'Cellular Technology', 'RAM', 'Product Dimensions', 'Batteries',
       'Item model number', 'Wireless communication technologies',
       'Connectivity technologies', 'GPS', 'Special features',
       'Display technology', 'Other display features', 'Resolution',
       'Other camera features', 'Audio Jack', 'Form factor', 'Colour',
       'Battery Power Rating', 'Whats in the box', 'Manufacturer',
       'Country of Origin', 'Item Weight', 'Screen Size', 'Screen Type',
       'Battery Power (In mAH)', 'Inbuilt Storage (in GB)', 'Processor Brand',
       'Operating System', 'Warranty Details', 'Camera Description',
       'Battery type', 'Processor Speed', 'Item Dimensions',
       'What's in the box', 'no of 5 star', 'no of 4 star', 'no of 3 star',
       'no of 2 star', 'no of 1 star', 'Device interface - primary',
       'Memory Storage Capacity', 'Phone Talk Time', 'Item part number',
       'Phone Standb

In [ ]:
na_counts = data.isna().sum().sort_values(ascending=False)
# Filter columns with NA > 100
filtered_columns = na_counts[na_counts > 100].index.tolist()
filtered_columns # to remove

['Rear Camera Resolution',
 'Battery Power',
 'Optical Sensor Resolution',
 'Wireless network technology',
 'Connectivity Technology',
 'Imported By',
 'GPU',
 'Item part number',
 'Colours displayed',
 'Package Dimensions',
 'Phone Talk Time',
 'Phone Standby Time (with data)',
 'Memory Storage Capacity',
 'Display technology',
 'Resolution',
 'Audio Jack',
 'Processor Brand',
 'Device interface - primary',
 'Processor Speed']

In [ ]:
data.drop(columns=filtered_columns, inplace=True)

In [ ]:
len(data.columns)

38

In [ ]:
# data.to_csv("../../data/processed.csv")
# data.to_json("../../data/processed.json", orient='records')

In [ ]:
data.isna().sum().sort_values(ascending=False)

Item model number                      87
Batteries                              78
Network Service Provider               49
Item Dimensions                        48
Product Dimensions                     48
GPS                                    45
Battery type                           35
Model Name                             19
Wireless communication technologies    14
Screen Size                            13
Form factor                            11
Warranty Details                       11
Country of Origin                       9
Cellular Technology                     7
Special features                        7
Screen Type                             6
Whats in the box                        5
Connectivity technologies               5
What's in the box                       5
Other camera features                   3
Camera Description                      3
Colour                                  2
Battery Power Rating                    2
Battery Power (In mAH)            

In [ ]:
def missing_values_intersection(columns):
    # Create boolean masks for each column
    isna_masks = [data[col].isna() for col in columns]

    # Count intersecting and separate missing values
    intersect_count = sum(all(isna_masks) for isna_masks in zip(*isna_masks))
    separate_count = sum(any(isna_masks) for isna_masks in zip(*isna_masks)) - intersect_count

    return intersect_count, separate_count

In [ ]:
missing_values_intersection(["Form factor", "Screen Size"])

(1, 22)

In [ ]:
missing_values_intersection(["Product Dimensions", "Item Dimensions"])

(48, 0)

In [ ]:
data["GPS"].value_counts()

GPS
GLONASS                                                                                 160
True                                                                                    135
GPS                                                                                      10
AGPS                                                                                      5
GPS/AGPS,Glonass, Beidou, Galileo                                                         4
GPS,Glonass,Beidou,Galileo                                                                4
GPS (L1 + L5), GLO (G1), BDS (B1i + B1c + B2a), GAL (E1 + E5a), QZSS (L1 + L5), AGPS      3
GPS/GNSS/BEIDOU/Galileo                                                                   1
GPS,Glonass,Beidou,Galileo,QZSS                                                           1
GPS/GNSS/BEIDOU/Galileo/QZSS                                                              1
Supported                                                                   

In [ ]:
to_remove = [
    "Batteries",
    "Item model number",
    "Product Dimensions", # item dim better
    "Operating System", # OS better,
    "Battery Power Rating", # in mAh better
    "Whats in the box", # i will use the proper english,
    "Other camera features", # i will use Camera Description
    "Country of Origin",
    "Other display features", # ITS ALL THE SAME
    "Model Name",
    "Brand"
]

In [ ]:
data.drop(columns=to_remove, inplace=True)

In [ ]:
data.isna().sum().sort_values(ascending=False)

Network Service Provider               49
Item Dimensions                        48
GPS                                    45
Battery type                           35
Wireless communication technologies    14
Screen Size                            13
Form factor                            11
Warranty Details                       11
Special features                        7
Cellular Technology                     7
Screen Type                             6
Connectivity technologies               5
What's in the box                       5
Camera Description                      3
Colour                                  2
RAM                                     2
Battery Power (In mAH)                  2
OS                                      2
Manufacturer                            1
Inbuilt Storage (in GB)                 1
no of 2 star                            0
no of 4 star                            0
no of 3 star                            0
mrp                               

In [ ]:
all_null = data.isna().sum().sort_values(ascending=False)
all_null = all_null[all_null > 0].index.tolist()
all_null

['Network Service Provider',
 'Item Dimensions',
 'GPS',
 'Battery type',
 'Wireless communication technologies',
 'Screen Size',
 'Form factor',
 'Warranty Details',
 'Special features',
 'Cellular Technology',
 'Screen Type',
 'Connectivity technologies',
 "What's in the box",
 'Camera Description',
 'Colour',
 'RAM',
 'Battery Power (In mAH)',
 'OS',
 'Manufacturer',
 'Inbuilt Storage (in GB)']

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
# join  and Connectivity technologies

In [ ]:
def convert_string_to_list(string):
    if pd.isnull(string):
        return list()
    else:
        string = string.replace(';', ' ').replace(',', ' ')
        string = string.lower()
        lst = string.split()
        return lst

# Convert the columns to lists
for column in ["Wireless communication technologies", "Connectivity technologies"]:
    data[column] = data[column].apply(convert_string_to_list)

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
column1, column2 = "Wireless communication technologies", "Connectivity technologies"

def combine_lists(row):
    # Combine the two lists and remove duplicates
    combined_list = list(set(row[column1] + row[column2]))
    return combined_list

# Create a new column with the combined lists
data['connectivity_tech'] = data.apply(combine_lists, axis=1)

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
data.drop(columns=[column1, column2], inplace=True)

In [ ]:
data["Network Service Provider"] = data["Network Service Provider"].str.replace("Unlocked for All Carriers", "ALL")
data["Network Service Provider"] = data["Network Service Provider"].str.replace("Unlocked", "ALL")
data["Network Service Provider"] = data["Network Service Provider"].fillna("ALL")

In [ ]:
data["Network Service Provider"].unique()

array(['ALL', 'Republic Wireless', 'Total Wireless',
       'VoiceStream Wireless Provider Type', '3', 'Vodafone', 'AT&T',
       'E-Plus'], dtype=object)

In [ ]:
data.isna().sum().sort_values(ascending=False)

Item Dimensions             48
GPS                         45
Battery type                35
Screen Size                 13
Warranty Details            11
Form factor                 11
Cellular Technology          7
Special features             7
Screen Type                  6
What's in the box            5
Camera Description           3
Battery Power (In mAH)       2
RAM                          2
OS                           2
Colour                       2
Manufacturer                 1
Inbuilt Storage (in GB)      1
no of 1 star                 0
no of 2 star                 0
no of 4 star                 0
no of 3 star                 0
mrp                          0
no of 5 star                 0
Network Service Provider     0
Item Weight                  0
connectivity_tech            0
dtype: int64

In [ ]:
data["Item Dimensions"].unique()

array(['0.9 x 16.5 x 7.7 cm', '16.6 x 7.6 x 0.8 cm',
       '16.4 x 7.6 x 0.8 cm', '16.4 x 7.5 x 0.8 cm', '16 x 7.3 x 0.8 cm',
       '16.6 x 0.8 x 7.6 cm', '16.4 x 0.9 x 7.6 cm',
       '18.4 x 9.8 x 6.5 cm', '0.9 x 7.7 x 16.7 cm',
       '16.5 x 0.9 x 7.7 cm', '14.8 x 7.2 x 0.9 cm', nan,
       '15 x 10 x 5 cm', '14.7 x 7.3 x 1 cm', '16.5 x 7.6 x 0.9 cm',
       '16.4 x 7.6 x 0.9 cm', '9.00 cm x 77.00 cm x 1.65 metres',
       '16.1 x 7.4 x 0.9 cm', '0.84 x 7.55 x 16.44 cm',
       '0.84 x 7.62 x 16.84 cm', '17.1 x 7.8 x 0.9 cm',
       '16.9 x 7.7 x 0.9 cm', '16.9 x 7.6 x 0.9 cm',
       '0.9 x 7.6 x 16.4 cm', '7.6 x 0.8 x 16.6 cm',
       '7.6 x 0.9 x 16.4 cm', '7.3 x 0.8 x 15.9 cm',
       '0.84 x 7.45 x 15.98 cm', '16 x 0.8 x 7.5 cm',
       '15.2 x 0.8 x 6.9 cm', '16.5 x 0.8 x 7.7 cm',
       '16.48 x 7.22 x 0.81 cm', '7.6 x 0.8 x 16.1 cm',
       '15.5 x 7.45 x 15.57 cm', '18 x 7 x 7 cm', '0.8 x 7.8 x 16.1 cm',
       '15.8 x 0.8 x 7.4 cm', '15 x 0.8 x 7 cm', '16.2 x 0.8 x 7.7 

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
data["Cellular Technology"].unique()

array(['5G', '4G', 'LTE', '2G,3 G,4G,LTE,VOLTE', '2G',
       '4G VOLTE, 4G, 3G, 2G', '2G, 3G, 4G', '4G,3G,2G', '5G, 4G LTE',
       '5G, 4G', '5G, GSM', '(4G+5G)', nan,
       '2G GSM,3G WCDMA,4G LTE FDD,4G LTE TDD', 'AMPS',
       'Supports 5G* / 4G / 3G/ 2G *Supported 5G Bands: NR (SA & NSA): n1/n3/n5/n7/n8/n20/n28/n38/n40/n41/n66/n71/n75/n77/n78/n79  See more',
       '2G,3 G,4G', '2G/3G/4G/LTE/VOLTE'], dtype=object)

In [ ]:
# data.reset_index().to_json("temp.json", orient="records")

In [ ]:
len(list(data.dropna(subset=["Item Dimensions"]).iterrows()))

323

In [ ]:
data.dropna(subset=["Item Dimensions"], inplace=True)

In [ ]:
data.isna().sum().sort_values(ascending=False)

GPS                         34
Battery type                21
Form factor                  9
Warranty Details             6
Cellular Technology          6
Screen Size                  6
Screen Type                  4
Special features             4
What's in the box            3
Battery Power (In mAH)       2
Colour                       1
OS                           1
Camera Description           1
no of 3 star                 0
no of 1 star                 0
no of 4 star                 0
no of 5 star                 0
no of 2 star                 0
mrp                          0
Item Dimensions              0
Inbuilt Storage (in GB)      0
Network Service Provider     0
Item Weight                  0
Manufacturer                 0
RAM                          0
connectivity_tech            0
dtype: int64

In [ ]:
to_remove = [
    "Colour",
]

data.drop(columns=to_remove, inplace=True)

In [ ]:
len(list(data[data.isnull().any(axis=1)].iterrows()))

66

In [ ]:
data.isna().sum().sort_values(ascending=False)

GPS                         34
Battery type                21
Form factor                  9
Screen Size                  6
Warranty Details             6
Cellular Technology          6
Screen Type                  4
Special features             4
What's in the box            3
Battery Power (In mAH)       2
OS                           1
Camera Description           1
no of 3 star                 0
no of 1 star                 0
no of 4 star                 0
no of 5 star                 0
no of 2 star                 0
mrp                          0
Item Dimensions              0
Inbuilt Storage (in GB)      0
Network Service Provider     0
Item Weight                  0
Manufacturer                 0
RAM                          0
connectivity_tech            0
dtype: int64

In [ ]:
len(data.columns)

25

In [ ]:
missing_values_intersection(["Warranty Details", "Camera Description", "What's in the box", "Special features"])

(1, 8)

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
data["Battery type"].value_counts()

Battery type
Lithium Ion          164
Lithium Polymer      133
Polymer                2
Lithium-Phosphate      1
Lithium                1
Lithium-Cobalt         1
Name: count, dtype: int64

In [ ]:
data["GPS"].value_counts()

GPS
True                                                                                    135
GLONASS                                                                                 129
GPS                                                                                       6
GPS/AGPS,Glonass, Beidou, Galileo                                                         4
GPS,Glonass,Beidou,Galileo                                                                4
AGPS                                                                                      3
GPS (L1 + L5), GLO (G1), BDS (B1i + B1c + B2a), GAL (E1 + E5a), QZSS (L1 + L5), AGPS      3
GPS/GNSS/BEIDOU/Galileo                                                                   1
GPS,Glonass,Beidou,Galileo,QZSS                                                           1
GPS/GNSS/BEIDOU/Galileo/QZSS                                                              1
Supported                                                                   

In [ ]:
mapping = {
    'GLONASS': 'GLONASS',
    'True': 'GPS',
    'GPS,Glonass,Beidou,Galileo': 'GPS/GLONASS/Beidou/Galileo',
    'GPS': 'GPS',
    'GPS/AGPS,Glonass, Beidou, Galileo': 'GPS/GLONASS/Beidou/Galileo',
    'GPS (L1 + L5), GLO (G1), BDS (B1i + B1c + B2a), GAL (E1 + E5a), QZSS (L1 + L5), AGPS': 'GPS/GLONASS/Beidou/Galileo/QZSS',
    'GPS,Glonass,Beidou,Galileo,QZSS': 'GPS/GLONASS/Beidou/Galileo/QZSS',
    'GPS/GNSS/Galileo/BeiDou': 'GPS/GNSS/Galileo/BeiDou',
    'AGPS': 'AGPS',
    'Supported': 'Supported',
    'GPS/AGPS, GLONASS,Galelio, BeiDou, QZSS': 'GPS/AGPS/GLONASS/Galileo/BeiDou/QZSS',
    'GPS/GNSS/BEIDOU/Galileo': 'GPS/GNSS/Beidou/Galileo',
    'GPS/GNSS/BEIDOU/Galileo/QZSS': 'GPS/GNSS/Beidou/Galileo/QZSS'
}

data['GPS'] = data['GPS'].replace(mapping)

In [ ]:
# data['GPS'] = data['GPS'].str.split('/')

In [ ]:
def split_cellular_tech(row):
    if pd.isnull(row["GPS"]):
        return list()
    else:
        lst = row["GPS"].split("/")
        return lst

data["GPS"] = data.apply(split_cellular_tech, axis=1)

In [ ]:
data["GPS"].value_counts()

GPS
[GPS]                                          141
[GLONASS]                                      129
[]                                              34
[GPS, GLONASS, Beidou, Galileo]                  8
[GPS, GLONASS, Beidou, Galileo, QZSS]            4
[AGPS]                                           3
[GPS, GNSS, Beidou, Galileo]                     1
[GPS, GNSS, Beidou, Galileo, QZSS]               1
[Supported]                                      1
[GPS, AGPS, GLONASS, Galileo, BeiDou, QZSS]      1
Name: count, dtype: int64

In [ ]:
data['GPS'].apply(pd.Series).stack().value_counts()

GPS          156
GLONASS      142
Galileo       15
Beidou        14
QZSS           6
AGPS           4
GNSS           2
Supported      1
BeiDou         1
Name: count, dtype: int64

In [ ]:
data.drop(columns=["GPS"], inplace=True)

In [ ]:
data.columns

Index(['mrp', 'Network Service Provider', 'OS', 'Cellular Technology', 'RAM',
       'Special features', 'Form factor', 'Manufacturer', 'Item Weight',
       'Screen Size', 'Screen Type', 'Battery Power (In mAH)',
       'Inbuilt Storage (in GB)', 'Warranty Details', 'Camera Description',
       'Battery type', 'Item Dimensions', 'What's in the box', 'no of 5 star',
       'no of 4 star', 'no of 3 star', 'no of 2 star', 'no of 1 star',
       'connectivity_tech'],
      dtype='object')

In [ ]:
data.isna().sum().sort_values(ascending=False)

Battery type                21
Form factor                  9
Warranty Details             6
Cellular Technology          6
Screen Size                  6
Special features             4
Screen Type                  4
What's in the box            3
Battery Power (In mAH)       2
OS                           1
Camera Description           1
mrp                          0
no of 3 star                 0
no of 4 star                 0
no of 2 star                 0
no of 1 star                 0
no of 5 star                 0
Inbuilt Storage (in GB)      0
Item Dimensions              0
Network Service Provider     0
Item Weight                  0
Manufacturer                 0
RAM                          0
connectivity_tech            0
dtype: int64

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
# data.drop('B09Y64H8VS', inplace=True)

In [ ]:
len(list(data.iterrows()))

323

In [ ]:
mapping = {
    'Samsung': 'Samsung',
    'Redmi': 'Xiaomi',
    'Xiaomi': 'Xiaomi',
    'Xiaomi Technology India Private Limited': 'Xiaomi',
    'Rising Stars Mobile India Private Limited': 'Xiaomi',
    'OPPO Mobiles India Pvt Ltd': 'OPPO',
    'Oppo Mobiles India Private Limited 5th Floor, Tower-B, Building No. 8, Haryana-122002, India': 'OPPO',
    'Oppo Mobiles India Private Limited': 'OPPO',
    'OPPO Mobiles India Private Limited': 'OPPO',
    'Lava': 'Lava',
    'LAVA': 'Lava',
    'G mobiles': 'G-Mobile',
    'G-MOBILE': 'G-Mobile',
    'G Mobiles': 'G-Mobile',
    'G-Mobile Devices Private Limited': 'G-Mobile',
    'S MOBILE DEVICES PRIVATE LTD': 'G-Mobile',
    '1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box': 'Generic',
    'Samsung India pvt Ltd': 'Samsung',
    'Samsung India Electronics Pvt ltd': 'Samsung',
    'Dixon Technologies (India) Ltd.,Plot No.6, Sector-90,Noida, Gautam Buddha Nagar, U.P. India-201305': 'Dixon Technologies',
    'OnePlus': 'OnePlus',
    'vivo Mobile India Pvt Ltd': 'Vivo',
    'Vivo': 'Vivo',
    'Vivo Mobile India Pvt Ltd': 'Vivo',
    'vivo': 'Vivo',
    'vivo Mobile India Private Limited': 'Vivo',
    'For and on behalf of HMD Mobile India Private Limited': 'Nokia',
    'Nokia': 'Nokia',
    'Bhagwati Products Ltd': 'Micromax',
    'MICROMAX': 'Micromax',
    'Micromax': 'Micromax',
    'Realme': 'Realme',
    'iQOO': 'iQOO',
    'Generic': 'generic'
}

data['Manufacturer'] = data['Manufacturer'].replace(mapping)

In [ ]:
for index, row in data[data['Battery type'].isnull()].iterrows():
    manufacturer = row['Manufacturer']
    manufacturer_df = data[data['Manufacturer'] == manufacturer]

    if len(manufacturer_df) <= 1:
        continue

    mode_result = manufacturer_df['Battery type'].mode()
    if mode_result.empty:
        continue

    most_common_screen_type = mode_result.values[0]
    data.at[index, 'Battery type'] = most_common_screen_type

In [ ]:
data['Screen Type'].isnull()

product_id
B09TWDYSWQ    False
B0BZ479WZD    False
B0BMQSF1M4    False
B07WJW747N    False
B0BMQS9NVT    False
B0B449C737    False
B082F2T5PQ    False
B0BQ3MMPX6    False
B09LHX1YFX    False
B09ZBF5Y4J    False
B0BZCSNTT4    False
B0C14MRRVN    False
B0BBH4C5KT    False
B0BBN3WF7V    False
B09GFLXVH9    False
B09FLYW7FG    False
B0BZ8W9XQW    False
B09S3W77PT    False
B09S3X9VXX    False
B0BJ2YKLHM    False
B0BNBZRRJM    False
B09JGCFLGY    False
B0B1QH8RMV    False
B09RV7G1XQ    False
B09RV75JJM    False
B08RWNQJVL    False
B09RV5YBJW    False
B08RDMBPDK    False
B091J15RHZ    False
B09RV799HF    False
B0BW5QRCPG    False
B0BXLPY5WW    False
B0BTLJM8MS    False
B0BRNM8ZPJ    False
B0C14P16QR    False
B0C14PV6WY    False
B0BX9YM275    False
B0C14LHTL7    False
B0BWSFX131    False
B09MWCWRMP    False
B0BX9BDG4Q    False
B0BZCSMP95    False
B0BYN48MQW    False
B0BMGB2TPR    False
B0BY8JZ22K    False
B09WQYFLRX    False
B0B3CPQ5PF    False
B08VB57558    False
B094W56F6B    False
B0893F6H3

In [ ]:
for index, row in data[data['Screen Type'].isnull()].iterrows():
    manufacturer_df = data[data['Manufacturer'] == row['Manufacturer']]

    if len(manufacturer_df) <= 1:
        continue

    mode_result = manufacturer_df['Screen Type'].mode()
    if mode_result.empty:
        continue

    most_common_screen_type = mode_result.values[0]
    data.at[index, 'Screen Type'] = most_common_screen_type

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
data.dropna(subset=["Battery type"], inplace=True)
data.drop(columns=["Manufacturer"], inplace=True)

In [ ]:
len(data.columns)

23

In [ ]:
data["Warranty Details"].value_counts()

Warranty Details
1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories including batteries from the date of purchase                                                                                                                                                     86
6-month seller warranty                                                                                                                                                                                                                                                                             29
1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box accessories                                                                                                                                                                                                   20
1 year manufacturer warranty for device and 6 months manufacturer warranty for in-box             

In [ ]:
def extract_warranty(row):
    warranty = row['Warranty Details']

    if pd.isnull(warranty):
        return [0, 0]
    
    # Extract the warranty duration for the phone
    phone_duration = re.findall(r'(\d+)\s*year', warranty)
    if phone_duration:
        phone_duration = int(phone_duration[0])
    else:
        phone_duration = 0
    
    # Extract the warranty duration for the items along with the phone
    items_duration = re.findall(r'(\d+)\s*months', warranty)
    if items_duration:
        items_duration = int(items_duration[0])
    else:
        items_duration = 0
    
    return [phone_duration, items_duration]

# Apply the extract_warranty function to the "Warranty Details" column
data[['Phone Warranty (months)', 'Items Warranty (months)']] = data.apply(extract_warranty, axis=1, result_type='expand')

In [ ]:
data.drop(columns=['Warranty Details', 'Screen Size'], inplace=True)

In [ ]:
len(data.columns)

23

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
len(list(data.iterrows()))

321

In [ ]:
data.isna().sum().sort_values(ascending=False)

Form factor                 9
Cellular Technology         6
Special features            4
What's in the box           3
Battery Power (In mAH)      2
Camera Description          1
OS                          1
Phone Warranty (months)     0
connectivity_tech           0
no of 1 star                0
no of 2 star                0
no of 3 star                0
no of 4 star                0
no of 5 star                0
mrp                         0
Item Dimensions             0
Battery type                0
Network Service Provider    0
Inbuilt Storage (in GB)     0
Screen Type                 0
Item Weight                 0
RAM                         0
Items Warranty (months)     0
dtype: int64

In [ ]:
data.dropna(subset=["Form factor"], inplace=True)

In [ ]:
data.isna().sum().sort_values(ascending=False)

Cellular Technology         5
Special features            3
Battery Power (In mAH)      2
What's in the box           2
mrp                         0
Item Dimensions             0
Phone Warranty (months)     0
connectivity_tech           0
no of 1 star                0
no of 2 star                0
no of 3 star                0
no of 4 star                0
no of 5 star                0
Camera Description          0
Battery type                0
Network Service Provider    0
Inbuilt Storage (in GB)     0
Screen Type                 0
Item Weight                 0
Form factor                 0
RAM                         0
OS                          0
Items Warranty (months)     0
dtype: int64

In [ ]:
data["Cellular Technology"].value_counts()

Cellular Technology
4G                                                                                                                                     126
5G                                                                                                                                     102
LTE                                                                                                                                     31
5G, 4G LTE                                                                                                                              10
2G,3 G,4G,LTE,VOLTE                                                                                                                      9
4G,3G,2G                                                                                                                                 7
2G                                                                                                                                       6
4G VOLT

In [ ]:
mapping = {
    'Supports 5G* / 4G / 3G/ 2G *Supported 5G Bands: NR (SA & NSA): n1/n3/n5/n7/n8/n20/n28/n38/n40/n41/n66/n71/n75/n77/n78/n79  See more': "5G / 4G / 3G/ 2G",
    "(4G,3G,2G)": "4G,3G,2G",
}

data['Cellular Technology'] = data['Cellular Technology'].replace(mapping)

In [ ]:
data['Cellular Technology'].value_counts()

Cellular Technology
4G                                       126
5G                                       102
LTE                                       31
5G, 4G LTE                                10
2G,3 G,4G,LTE,VOLTE                        9
4G,3G,2G                                   7
2G                                         6
4G VOLTE, 4G, 3G, 2G                       3
2G, 3G, 4G                                 3
5G, 4G                                     2
5G, GSM                                    2
2G GSM,3G WCDMA,4G LTE FDD,4G LTE TDD      2
(4G+5G)                                    1
5G / 4G / 3G/ 2G                           1
2G,3 G,4G                                  1
2G/3G/4G/LTE/VOLTE                         1
Name: count, dtype: int64

In [ ]:
def split_cellular_tech(row):
    if pd.isnull(row["Cellular Technology"]):
        return None
    else:
        string = row["Cellular Technology"].replace(",", "/")
        string = string.replace("*", "/")
        lst = [char.strip() for char in string.split("/")]
        return lst

data["Cellular Technology"] = data.apply(split_cellular_tech, axis=1)

In [ ]:
data["Cellular Technology"].value_counts()

Cellular Technology
[4G]                                          126
[5G]                                          102
[LTE]                                          31
[5G, 4G LTE]                                   10
[2G, 3 G, 4G, LTE, VOLTE]                       9
[4G, 3G, 2G]                                    7
[2G]                                            6
[4G VOLTE, 4G, 3G, 2G]                          3
[2G, 3G, 4G]                                    3
[5G, 4G]                                        2
[5G, GSM]                                       2
[2G GSM, 3G WCDMA, 4G LTE FDD, 4G LTE TDD]      2
[(4G+5G)]                                       1
[5G, 4G, 3G, 2G]                                1
[2G, 3 G, 4G]                                   1
[2G, 3G, 4G, LTE, VOLTE]                        1
Name: count, dtype: int64

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
data["Cellular Technology"].apply(pd.Series).stack().value_counts()

4G            153
5G            117
LTE            41
2G             31
3G             15
3 G            10
VOLTE          10
4G LTE         10
4G VOLTE        3
GSM             2
2G GSM          2
3G WCDMA        2
4G LTE FDD      2
4G LTE TDD      2
(4G+5G)         1
Name: count, dtype: int64

In [ ]:
def replace_cellular_technology(lst):
    if lst is None:
        return None

    replacements = {
        '4G LTE FDD': '4G LTE',
        '4G LTE TDD': '4G LTE',
        '3G WCDMA': '3G',
        '3 G': '3G',
        '4G LTE': '4G',
        'GSM': '2G',
        "LTE": "4G",
        "2G GSM": "2G",
        "4G VOLTE": "4G",
        "VOLTE": "4G"
    }

    return [replacements.get(val, val) for val in lst]

data['Cellular Technology'] = data['Cellular Technology'].apply(replace_cellular_technology)

In [ ]:
data[data.isnull().any(axis=1)].to_csv("temp.csv")

In [ ]:
len(list(data.iterrows()))

312

In [ ]:
data.dropna(subset=["Battery Power (In mAH)", "Cellular Technology", "Camera Description"], inplace=True)

In [ ]:
len(list(data.iterrows()))

305

In [ ]:
data.isna().sum().sort_values(ascending=False)

Special features            3
mrp                         0
Battery type                0
Phone Warranty (months)     0
connectivity_tech           0
no of 1 star                0
no of 2 star                0
no of 3 star                0
no of 4 star                0
no of 5 star                0
What's in the box           0
Item Dimensions             0
Camera Description          0
Network Service Provider    0
Inbuilt Storage (in GB)     0
Battery Power (In mAH)      0
Screen Type                 0
Item Weight                 0
Form factor                 0
RAM                         0
Cellular Technology         0
OS                          0
Items Warranty (months)     0
dtype: int64

In [ ]:
len(list(data[data.isnull().any(axis=1)].iterrows()))

3

In [ ]:
data[data.notna().any(axis=1)].to_csv("temp.csv")

In [ ]:
def get_num_from_bat(row):
    if pd.isnull(row["Battery Power (In mAH)"]):
        return None
    else:
        string = row["Battery Power (In mAH)"].split()[0]
        return string

data["Battery Power (In mAH)"] = data.apply(get_num_from_bat, axis=1).astype(int)

In [ ]:
def get_num_from_ram(row):
    if pd.isnull(row["RAM"]):
        return None
    else:
        string = str(row["RAM"]).split()[0]
        return string

data["RAM"] = data.apply(get_num_from_ram, axis=1).astype(int)

ValueError: invalid literal for int() with base 10: '0.05'

In [ ]:
data["RAM"].value_counts()

RAM
8      98
4      62
6      39
3      18
2      13
12     10
128     5
64      4
256     2
16      2
Name: count, dtype: int64

In [ ]:
def get_num_from_inbuilt(row):
    if pd.isnull(row["Inbuilt Storage (in GB)"]):
        return None
    else:
        string = str(row["Inbuilt Storage (in GB)"]).split()[0]
        return string

data["Inbuilt Storage (in GB)"] = data.apply(get_num_from_inbuilt, axis=1).astype(int)

In [ ]:
def get_num_from_gram(row):
    if pd.isnull(row["Item Weight"]):
        return None
    else:
        string, type = str(row["Item Weight"]).split()
        if type == "kg":
            string = float(string) * 1000
        string = int(float(string))
        return string

data["Item Weight"] = data.apply(get_num_from_gram, axis=1).astype(int)

In [ ]:
def extract_dimensions(row):
    dimensions = str(row["Item Dimensions"])
    seperations = [value.split()[0] for value in dimensions.split(" x ")]

    length, width, height = sorted(map(
        float, seperations
    ), reverse=True)

    if height > 10:
        height /= 10
    
    if width > 10:
        width /= 10
    
    return [length, width, height]

# for index, row in data.iterrows():
#     print(index, str(row["Item Dimensions"]))

data[['length', 'width', 'height']] = data.apply(extract_dimensions, axis=1, result_type='expand')

In [ ]:
data[data.notna().any(axis=1)].to_csv("temp.csv")

In [ ]:
data["Camera Description"].value_counts()

Camera Description
Rear, Front                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   53
Front                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
def camera_features(desc):
    features = {}

    # Number of cameras
    if 'quad' in desc.lower():
        features['camera_count'] = 4
    elif 'triple' in desc.lower():
        features['camera_count'] = 3
    elif 'dual' in desc.lower():
        features['camera_count'] = 2
    else:
        features['camera_count'] = 1  # default is one camera

    # Front and Rear camera
    if 'front' in desc.lower():
        features['has_front_camera_details'] = int('mp' in desc.lower())
    else:
        features['has_front_camera_details'] = 0  # default is no specifics 

    if 'rear' in desc.lower():
        features['has_rear_camera_details'] = int('mp' in desc.lower())
    else:
        features['has_rear_camera_details'] = 0  # default is no specifics 

    # Check for specific features
    features['cam_has_AI'] = int('ai' in desc.lower() or features['has_rear_camera_details'])
    features['cam_has_OIS'] = int('ois' in desc.lower() or features['has_rear_camera_details'])
    features['cam_has_Zoom'] = int('zoom' in desc.lower() or features['has_rear_camera_details'])
    features['cam_has_HDR'] = int('hdr' in desc.lower() or features['has_rear_camera_details'])
    features['cam_has_Macro'] = int('macro' in desc.lower() or features['has_rear_camera_details'])
    features['cam_has_Portrait'] = int('portrait' in desc.lower() or features['has_rear_camera_details'])

    # Camera resolution
    match = re.search(r'(\d+)MP', desc)
    if match:
        features['main_camera_MP'] = int(match.group(1))
    else:
        features['main_camera_MP'] = 12  # default is 12MP

    del features["has_rear_camera_details"], features['has_front_camera_details']

    return pd.Series(features)

camera_features_df = data['Camera Description'].apply(camera_features)
data = pd.concat([data, camera_features_df], axis=1)

In [ ]:
data.drop(columns=["Camera Description", "Item Dimensions", "What's in the box", "Special features"], inplace=True)

In [ ]:
len(data.columns)

29

In [ ]:
data[data.notna().any(axis=1)].to_csv("temp.csv")

In [ ]:
data[data.notna().any(axis=1)].to_csv("temp.csv")

In [ ]:
data["Network Service Provider"].value_counts()

Network Service Provider
ALL                                   247
Total Wireless                          3
VoiceStream Wireless Provider Type      1
AT&T                                    1
Vodafone                                1
Name: count, dtype: int64

In [ ]:
data.drop(columns=["Network Service Provider"], inplace=True)

In [ ]:
data.columns

Index(['OS', 'Cellular Technology', 'RAM', 'Form factor', 'Item Weight',
       'Screen Type', 'Battery Power (In mAH)', 'Inbuilt Storage (in GB)',
       'Battery type', 'no of 5 star', 'no of 4 star', 'no of 3 star',
       'no of 2 star', 'no of 1 star', 'connectivity_tech',
       'Phone Warranty (months)', 'Items Warranty (months)', 'length', 'width',
       'height', 'camera_count', 'cam_has_AI', 'cam_has_OIS', 'cam_has_Zoom',
       'cam_has_HDR', 'cam_has_Macro', 'cam_has_Portrait', 'main_camera_MP'],
      dtype='object')

In [ ]:
replacements = { 
    # "Android 12.0": ,
    # "Android 11.0": ,
    # "Android 10.0": ,
    # "Android 13.0": ,
    # "Android": ,
    # "Android 9.0": ,
    # "OxygenOS": ,
    # "MIUI 13": ,
    # "FunTouch OS 12": ,
    # "Android 8.1": ,
    "HiOS 7.6 based on Android 11": "HiOS 7.6",
    "Android 12(Go edition)": "Android 12, go edition",
    "HiOS 8.6 based on Android 12": "HiOS 8.6",
    "Funtouch OS 12 based on Android 12": "Funtouch OS 12",
    "Funtouch OS 12 (Based on Android 11)": "Funtouch OS 12",
    "MIUI 13, Android 12.0": "MIUI 13",
    "Funtouch OS 12 Based On Android 12": "Funtouch OS 12",
    # "Funtouch OS 12": ,
    "MIUI 13, Android 12": "MIUI 13",
    "Funtouch OS 13 Based On Android 13": "Funtouch OS 13",
    "MIUI 14, Android 13.0": "MIUI 14",
    "MIUI 12, Android 11.0": "MIUI 12",
    "MIUI 12.5, Android MIUI 12.5": "MIUI 12.5",
    "Android 11 MIUI 12.5 on, 3 years of Android updates, MIUI 12.5 on Android 11, 3 years of Android updates": "MIUI 12.5",
    # "Android 8.0": ,
    # "MIUI 12": ,
    "Android 11 - MiUI 12.5, MiUI 12.5 (Android 11)": "MiUI 12.5",
    "Android 11 Stock": "Android 11.0",
    "Funtouch OS 11 (Based on Android 11)": "Funtouch OS 11",
    "Funtouch OS 13 based on Android 13": "Funtouch OS 13",
    "Android 11, Funtouch OS 11.1": "Funtouch OS 11",
    "Go Edition, Android 11.0": "Android 11, go edition",
    "Funtouch OS 12 (Based on Android 12)": "Funtouch OS 12",
    # "MIUI 12.5": ,
    "MIUI 12, Android 10.0": "MIUI 12",
    "Android 11.1 based Funtouch OS 11.1": "Funtouch OS 11.1",
    "Android 11 - Funtouch OS 11.1, Funtouch OS 11.1": "Funtouch OS 11.1",
    "Android 11 MIUI 12.5": "MIUI 12.5",
    "HiOS 8.0 based on Android 11, Android 10.0": "HiOS 8.0",
    "HiOS 12.0 based on Android 12": "HiOS 12.0",
    # "Windows 11 Home": ,
    "Android 10 Go Edition": "Android 10, go edition"
}

data["OS"] = data['OS'].replace(replacements)

In [ ]:
data["OS"].value_counts()

OS
Android 12.0              60
Android 11.0              45
Android 10.0              18
Android 13.0              17
Android                   16
MIUI 13                   14
Android 9.0               13
OxygenOS                  12
FunTouch OS 12            10
Funtouch OS 12             9
Android 8.1                8
MIUI 12.5                  4
HiOS 7.6                   4
HiOS 8.6                   3
MIUI 12                    3
Android 12, go edition     3
Funtouch OS 11.1           2
Funtouch OS 11             2
Funtouch OS 13             2
MiUI 12.5                  1
Android 8.0                1
MIUI 14                    1
Windows 11 Home            1
Android 11, go edition     1
HiOS 12.0                  1
HiOS 8.0                   1
Android 10, go edition     1
Name: count, dtype: int64

In [ ]:
def extract_os_info(row):
    os_info = str(row['OS'])
    name = os_info.split()[0].lower()
    go_edition = int("go edition" in os_info)
    try:
        version = float(os_info.replace(',', '').split()[1])
    except:
        version = np.inf
    
    return name, version, go_edition


data[['os_name', 'os_version', 'os_go?']] = data.apply(extract_os_info, axis=1, result_type='expand')

In [ ]:
data.drop(columns=["OS"], inplace=True)

In [ ]:
data[data.notna().any(axis=1)].to_csv("temp.csv")